# Writing TIFF multilayer file readable from PhotoShop

There is no straightforward way to write a multilayer TIFF file readable from PhotoShop.
So far, the attempts in the following code have been unsuccessful.
* [This is a short descriotion of PS TIFF format](https://www.amyspark.me/blog/posts/2021/11/14/reading-and-writing-tiff-psds.html)
* [This is a thread with attempts to write a PS-readable TIFF](https://github.com/imageio/imageio/issues/708)

The following example works:
* [Layered TIFF with psdtags](https://github.com/cgohlke/psdtags/blob/master/examples/layered_tiff.py)

In [2]:
import tifffile
import numpy as np

In [3]:
import imagecodecs
import numpy
import tifffile
from psdtags import (
    PsdBlendMode,
    PsdChannel,
    PsdChannelId,
    PsdClippingType,
    PsdColorSpaceType,
    PsdCompressionType,
    PsdEmpty,
    PsdFilterMask,
    PsdFormat,
    PsdKey,
    PsdLayer,
    PsdLayerFlag,
    PsdLayerMask,
    PsdLayers,
    PsdRectangle,
    PsdString,
    PsdUserMask,
    TiffImageSourceData,
    __version__,
    overlay,
)

path = "../test/img-tif"
in_paths = [(path + "/000{:d}.tif".format(i)) for i in range(5)] + ["../test/img-tif-stack/0001_pyr.tif"]
images = [tifffile.imread(p) for p in in_paths]

# canvas height and width
shape = images[0].shape[:2]
transp = np.full_like(images[0][..., 0], 65535)
transp_1 = np.expand_dims(transp, axis=-1)
layers = [PsdLayer(
                name='Layer {}'.format(i),
                rectangle=PsdRectangle(0, 0, *shape),
                channels=[
                    PsdChannel(
                        channelid=PsdChannelId.TRANSPARENCY_MASK,
                        compression=PsdCompressionType.RAW,
                        data=transp,
                    ),
                    PsdChannel(
                        channelid=PsdChannelId.CHANNEL0,
                        compression=PsdCompressionType.RAW,
                        data=image[..., 0],
                    ),
                    PsdChannel(
                        channelid=PsdChannelId.CHANNEL1,
                        compression=PsdCompressionType.RAW,
                        data=image[..., 1],
                    ),
                    PsdChannel(
                        channelid=PsdChannelId.CHANNEL2,
                        compression=PsdCompressionType.RAW,
                        data=image[..., 2],
                    ),
                ],
                mask=PsdLayerMask(),
                opacity=255,
                blendmode=PsdBlendMode.NORMAL,
                blending_ranges=(),
                clipping=PsdClippingType.BASE,
                flags=PsdLayerFlag.PHOTOSHOP5,
                info=[
                    PsdString(PsdKey.UNICODE_LAYER_NAME, 'layer {}'.format(i)),
                ],
            ) for i, image in enumerate (images)]

In [4]:
# create the ImageSourceData structure for the layered TIFF
image_source_data = TiffImageSourceData(
    name='Layered TIFF',
    psdformat=PsdFormat.LE32BIT,
    layers=PsdLayers(
        key=PsdKey.LAYER,
        has_transparency=False,
        layers=layers,
    ),
    usermask=PsdUserMask(
        colorspace=PsdColorSpaceType.RGB,
        components=(65535, 0, 0, 0),
        opacity=50,
    ),
    info=[
        PsdEmpty(PsdKey.PATTERNS),
        PsdFilterMask(
            colorspace=PsdColorSpaceType.RGB,
            components=(65535, 0, 0, 0),
            opacity=50,
        ),
    ],
)

In [5]:
composite = overlay(*((np.concatenate((image, transp_1), axis=-1), (0, 0)) for image in images),
    shape=shape,
)

In [6]:
# write a layered TIFF file
tifffile.imwrite(
    'img-test/layered.tif',
    # write composite as main TIFF image, accessible to regular TIF readers
    composite,
    photometric='rgb',
    compression='adobe_deflate',
    # 72 dpi resolution
    resolution=((720000, 10000), (720000, 10000)),
    resolutionunit='inch',
    # do not write tifffile specific metadata
    metadata=None,
    # write layers and sRGB profile as extra tags
    extratags=[
        # ImageSourceData tag; use multiple threads to compress channels
        image_source_data.tifftag(maxworkers=4),
        # InterColorProfile tag
        (34675, 7, None, imagecodecs.cms_profile('srgb'), True),
    ],
)

In [7]:
isd = TiffImageSourceData.fromtiff('img-test/layered.tif')
print(isd)
print(f'psdtags {__version__}')

ValueError: cannot reshape array of size 5200000 into shape (1300,2000)

In [ ]:
# plot the layer and composite images in the TIFF file
for layer in isd.layers:
    tifffile.imshow(layer.asarray(), title=layer.name)
tifffile.imshow(tifffile.imread('img-test/layered.tif'), title='Composite', show=True)